In [1]:
import pandas as pd
import matplotlib

In [14]:
data = pd.read_csv('data/train_data/train_task_3_4.csv')
print(len(data))
data.head()

1382727


,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue
0,898,2111,280203,1,2,2
1,767,3062,55638,1,3,3
2,165,1156,386475,1,2,2
3,490,1653,997498,1,4,4
4,298,3912,578636,1,3,3


In [15]:
answer_metadata = pd.read_csv('data/metadata/answer_metadata_task_3_4.csv')
answer_metadata.head()
# need pre-processing in Confidence

,AnswerId,DateAnswered,Confidence,GroupId,QuizId,SchemeOfWorkId
0,1451945,2019-10-30 14:34:00.000,NaN,4,32,52562.0
1,45325,2020-01-06 18:53:00.000,75.0,185,66,52562.0
2,687013,2020-01-18 10:52:00.000,NaN,235,64,52562.0
3,91254,2020-02-29 17:25:00.000,NaN,194,97,52562.0
4,1225855,2020-03-06 15:07:00.000,NaN,95,115,52562.0


In [17]:
answer_metadata['Confidence'].fillna(value=answer_metadata['Confidence'].mean(), inplace = True)
answer_metadata.head()

,AnswerId,DateAnswered,Confidence,GroupId,QuizId,SchemeOfWorkId
0,1451945,2019-10-30 14:34:00.000,73.608704,4,32,52562.0
1,45325,2020-01-06 18:53:00.000,75.000000,185,66,52562.0
2,687013,2020-01-18 10:52:00.000,73.608704,235,64,52562.0
3,91254,2020-02-29 17:25:00.000,73.608704,194,97,52562.0
4,1225855,2020-03-06 15:07:00.000,73.608704,95,115,52562.0


In [18]:
# data left join with answer_metadata
data = data.merge(answer_metadata, on = "AnswerId", how = "left")
data.head()

,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue,DateAnswered,Confidence,GroupId,QuizId,SchemeOfWorkId
0,898,2111,280203,1,2,2,2019-12-08 17:47:00.000,73.608704,95,86,52562.0
1,767,3062,55638,1,3,3,2019-10-27 20:54:00.000,25.000000,115,39,52562.0
2,165,1156,386475,1,2,2,2019-10-06 20:16:00.000,73.608704,101,39,52562.0
3,490,1653,997498,1,4,4,2020-02-27 17:40:00.000,73.608704,46,115,52562.0
4,298,3912,578636,1,3,3,2019-12-27 16:07:00.000,73.608704,314,78,52562.0


In [20]:
data = data.groupby("QuestionId")[["Confidence", "IsCorrect"]].mean()
data

,Confidence,IsCorrect
QuestionId,,
0,70.912801,0.443457
1,73.608704,0.571429
2,71.716035,0.385214
3,76.624144,0.808757
4,71.122328,0.401408
...,...,...
943,77.013936,0.566528
944,72.933064,0.142857
945,73.248597,0.422336


In [35]:
data["Quality"] = data['IsCorrect'] * data['Confidence']
data


,QuestionId,Confidence,IsCorrect,Quality
0,944,72.933064,0.142857,10.419009
1,931,73.990989,0.160400,11.868120
2,155,74.523667,0.164776,12.279721
3,425,73.662215,0.179487,13.221423
4,718,72.669757,0.183891,13.363284
...,...,...,...,...
943,825,81.458674,0.888384,72.366552
944,841,73.608704,1.000000,73.608704
945,847,73.608704,1.000000,73.608704
946,660,73.608704,1.000000,73.608704


In [22]:
# Sort by ascending order of the 'quality' column.
data = data.sort_values("Quality", ascending = True).reset_index()
data

,QuestionId,Confidence,IsCorrect,Quality
0,944,72.933064,0.142857,10.419009
1,931,73.990989,0.160400,11.868120
2,155,74.523667,0.164776,12.279721
3,425,73.662215,0.179487,13.221423
4,718,72.669757,0.183891,13.363284
...,...,...,...,...
943,825,81.458674,0.888384,72.366552
944,841,73.608704,1.000000,73.608704
945,847,73.608704,1.000000,73.608704
946,660,73.608704,1.000000,73.608704


In [30]:
submission = pd.read_csv('data/submission/template.csv')
for i in range(len(data)):
    index = data[data['QuestionId'] == i].index
    submission.loc[i, 'ranking'] = index[0]
submission['ranking'] = submission['ranking'].astype('int')
submission.to_csv('data/submission/20172656.csv')